Vision transformer for OSR - eval

In cases where we want to classify OOD samples as such yet still perform multi-class classification over the ID samples, we call it OSR "open-set recognition". If we perform binary classification between ID and OOD, we call it OOD detection.

To create a OSR benchmark, given a test set of e.g. 10 classes for CIFAR10, we define splits CIFAR6 (.7 train ID, .2 val ID, .1 test ID) then CIFAR4 we define splits (OOD test split), with the goal of performing classification of the C legitimate classes, plus rejection of unknown classes.

oftentimes, OOD benchmarks involve an ID set, then several datasets with no class overlap to be used as OOD. (Filter if necessary).

We introduce CUB-200-2011 as an additional OOD benchmark?

Our CUB-200-2011 OSR benchmark creates train, val, test splits from CUB150, and reserves a CUB50 split for OSR testing (We expect 150-way classification plus rejection of the 50 classes not seen during training) This is challenging and interesting because the 50 unseen classes have significant fine-grained feature overlap with the 150 known classes.

In OOD there is the harder challenge of near-OOD: rejection of semantically mistakable samples. Going Beyond near-OOD, our paper investigates fine-grained OSR. First we establish that fine-grained OSR is indeed more difficult: CIFAR6/4 performance is reported as TODO. In contrast, a ResNet18 on CUB150/50 achieves only TODO. We hypothesize that (a) the greater inter-class similarity presents a considerable OSR challenge leading to degraded performance. Due to parameter sharing, ResNet excels at capturing local features efficiently. Yet distinguishing samples with highly similar features (such as images of different species of gull) requires a model that can learn nuanced fine-grained features.

Utilizing self-supervised learning over massive unlabeled datasets, Vision Transformers (ViT) surpass ConvNets across several computer vision benchmarks including OSR (Cai et al).

That paper evaluates on far fewer OOD methods than are included in OpenOOD. Therefore in this paper, we contribute
1. a more comprehensive evaluation of ViTs on the OSR task, by **evaluating a ViT against all post-hoc methods in OpenOOD**.

**Limitations** Cai et al also admit their results depend upon on dataset having labels. In our paper we assess fine-tuning a ViTb14 model (intractable?) on CUB by first throwing away the labels. We use Cai's supervised method as a baseline, then investigate whether similar results can be obtained through self-supervision (generated preannotations?).

2. self-supervised fine-tuning on the CUB dataset -> better fine-grained features ---linear---> logits --softmax--> "probs" ---( **many post hoc methods** )---> ood result.

Following Cai et al, we implement a ViT-based OSR detector consisting of the transformer-based backbone followed by a linear classification head. $W_f$, $W_c$, $W_d$ are all learned.

We negative-minimize the sum of probabilities that a classification is correct, on a log scale. Closed-set supervised training is now complete. After learning $W_f$, $W_c$ in the closed-set classification objective, we construct the novelty-detection objective function. This does NOT mean we are training on the OOD classes, which would not be OSR!! We fix $W_f$, $W_c$ to maintain accuracy for closed-set classes. Just as with the classification head, we take the features as input, but we declare $W_d$ and train a separate novelty detection module $\phi_d(\cdot;W_d)$.

Q: So what does the forward pass look like in a similarity-based method? *min (x-y) for all class centers y, so O(C).*

Post-hoc OOD methods can then be performed.


$$f: x \mapsto f \coloneqq \phi_f(x; W_f)$$

and

$$\phi_c(f; W_c)$$

where $f \in \mathbb{R^D}$ and $x \in \mathbb{R^{H x W x C}}$.

2. our **implementation of the Cai et al detector** to the OpenOOD upstream repository.

Q: BUT WAIT HOW CAN OPEN SET TRAINING RESULT IN A OSR EVAL?  *ViT params are fixed. Minibatch SGD with batch_size=256, lr=0.01, momenum=0.9.*

Q: BUT WAIT WHAT MODEL TYPE IS THE 2ND CLASSIFICATION HEAD? *A (768,C) linear detection head.*

In the CUB dataset, all classes are species of bird, such that CUB consists of subclasses of a single class, making it ideal as a fine-grained OSR benchmark.


Ok so:
1. is it viable to implement the cai detector then evaluate against all the openood postprocessing methods?
    or **does openood offer a readymade vit for OOD/OSR eval?**
2. viable to do self-supervised finetuning on dinov2?